In [2]:
from quoptuna.backend.data_processing.prepare import DataPreparation
from quoptuna import Optimizer


/Users/edwinjose/Documents/GitHub/quoptuna/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-20 16:03:11.055 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 16:03:11.056 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 16:03:11.198 
  command:

    streamlit run /Users/edwinjose/Documents/GitHub/quoptuna/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-20 16:03:11.198 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
import pandas as pd
import os

def mock_csv_data(data, tmp_path, file_name=None):

    dataframe = pd.DataFrame(data)
    file_name = file_name if file_name else "mock_csv"
    file_path = os.path.join(tmp_path, f"{file_name}.csv")
    dataframe.to_csv(file_path, index=False)
    return file_path

In [4]:
from pmlb import fetch_data, classification_dataset_names

# filtering Datasets with two classes

In [5]:
import pickle
with open("data/binary_datasets.pkl", "rb") as file:
    binary_datasets = pickle.load(file)

In [6]:
num = 101

# filter datatasets in binary_datasets where the second value in tuple is less than num
binary_datasets_filtered = [
    dataset for dataset in binary_datasets if dataset[1] < num
]
binary_datasets_filtered

[('analcatdata_aids', 50),
 ('analcatdata_asbestos', 83),
 ('analcatdata_bankruptcy', 50),
 ('analcatdata_creditscore', 100),
 ('analcatdata_cyyoung8092', 97),
 ('analcatdata_cyyoung9302', 92),
 ('analcatdata_fraud', 42),
 ('analcatdata_japansolvent', 52),
 ('labor', 57),
 ('lupus', 87),
 ('parity5', 32),
 ('postoperative_patient_data', 88)]

In [7]:
binary_datasets_filtered[-1:]

[('postoperative_patient_data', 88)]

In [8]:
def optimize_dataset(dataset_name):
    data = fetch_data(dataset_name, return_X_y=False)
    file_path = mock_csv_data(data,
                              tmp_path="data",
                              file_name=dataset_name)
    if "target" in data.columns:
        data_prep = DataPreparation(file_path=file_path,
                                    x_cols=list(data.columns.difference(["target"])),
                                    y_col="target")

        x, y = data_prep.select_columns()
        x_train, x_test, y_train, y_test = data_prep.prepare_data()
        data_dict = {
            "train_x": x_train,
            "test_x": x_test,
            "train_y": y_train,
            "test_y": y_test,
        }

        optimizer = Optimizer(db_name=f"dec_18_{dataset_name}",
                              study_name=dataset_name,
                              data=data_dict)
        study, best_trials = optimizer.optimize(n_trials=100)
    else:
        print("No target column found in the dataset")
        print(dataset_name)


In [9]:
import logging
import os
import sys

# Set up logging configuration
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s",
                    filename="optimize_log.txt",
                    filemode="a")

# ... existing code ...

for dataset in binary_datasets_filtered[-1:]:
    try:
        # Redirect stdout and stderr to the log file
        with open("optimize_log.txt", "a") as log_file:
            sys.stdout = log_file
            sys.stderr = log_file
            optimize_dataset(dataset[0])
            sys.stdout = sys.__stdout__
            sys.stderr = sys.__stderr__
    except Exception as e:
        logging.exception(f"Error processing dataset {dataset[0]}: {e}")

[I 2024-12-20 16:03:25,011] A new study created in RDB with name: postoperative_patient_data
[I 2024-12-20 16:04:16,417] Trial 0 finished with value: 0.7647058823529411 and parameters: {'max_vmap': 1, 'batch_size': 32, 'learning_rate': 0.001, 'n_input_copies': 2, 'n_layers': 5, 'observable_type': 'single', 'repeats': 10, 'C': 1, 'gamma_factor': 10, 'trotter_steps': 3, 't': 1.0, 'n_qfeatures': 'full', 'n_episodes': 10, 'visible_qubits': 'half', 'temperature': 100, 'encoding_layers': 1, 'degree': 2, 'n_qchannels': 5, 'qkernel_shape': 3, 'kernel_shape': 5, 'filter_name': 'smooth', 'gamma': 0.001, 'alpha': 0.001, 'hidden_layer_sizes': '(50, 10, 5)', 'eta0': 10, 'model_type': 'DressedQuantumCircuitClassifierSeparable'}. Best is trial 0 with value: 0.7647058823529411.
[I 2024-12-20 16:04:23,336] Trial 1 finished with value: 0.8333333333333334 and parameters: {'max_vmap': 1, 'batch_size': 32, 'learning_rate': 0.01, 'n_input_copies': 1, 'n_layers': 10, 'observable_type': 'half', 'repeats': 5, 

In [18]:
import logging
import os
import sys

# Set up logging configuration
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s",
                    filename="optimize_log.txt",
                    filemode="a")

# ... existing code ...

for dataset in binary_datasets_filtered:
    try:
        optimize_dataset(dataset[0])
    except Exception as e:
        logging.exception(f"Error processing dataset {dataset[0]}: {e}")

analcatdata_aids
analcatdata_asbestos
analcatdata_bankruptcy
analcatdata_creditscore
analcatdata_cyyoung8092
analcatdata_cyyoung9302
analcatdata_fraud
analcatdata_japansolvent
labor
lupus
parity5
postoperative_patient_data


In [5]:
from quoptuna import DataPreparation
# from quoptuna import mock_csv_data
from quoptuna import Optimizer

ImportError: cannot import name 'DataPreparation' from 'quoptuna' (/Users/edwinjose/Documents/GitHub/quoptuna/src/quoptuna/__init__.py)

In [1]:
from quoptuna.backend.data_processing.prepare import DataPreparation
from quoptuna.backend.data import mock_csv_data
from quoptuna import Optimizer

dataset_name = <Name of the dataset from pmlb>
data = fetch_data(dataset_name, return_X_y=False)
file_path = mock_csv_data(data,
                            tmp_path="data",
                            file_name=dataset_name)
if "target" in data.columns:
    data_prep = DataPreparation(file_path=file_path,
                                x_cols=list(data.columns.difference(["target"])),
                                y_col="target")

    x, y = data_prep.select_columns()
    x_train, x_test, y_train, y_test = data_prep.prepare_data()
    data_dict = {
        "train_x": x_train,
        "test_x": x_test,
        "train_y": y_train,
        "test_y": y_test,
    }

    optimizer = Optimizer(db_name=f"dec_18_{dataset_name}",
                            study_name=dataset_name,
                            data=data_dict)
    study, best_trials = optimizer.optimize(n_trials=100)

NameError: name 'fetch_data' is not defined